In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

import matplotlib.pyplot as plt
import bokeh.io
import bokeh.mpl
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
import seaborn as sns
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

In [3]:
import random

def points(num=10**4):

    def remdups(idx):
        i = 1
        while True:
            try:
                while p[i][idx] == p[i+1][idx]:
                    del p[i+1]
            except IndexError:
                break
            i += 1

    assert num > 1, 'num must be larger than 1'

    p = [(random.randint(1, 100000), random.randint(1, 100000)) for _ in range(num)]
    p.sort()
    remdups(0)
    p.sort(key=lambda x: x[1])
    remdups(1)

    py = p.copy()
    p.sort()

    return p, py

In [4]:
def closest_pair(px, py):

    def d(p1, p2):
        """Euclidean distance function"""
        return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)**0.5

    def closest_split_pair(pivot, delta):
        """Finds the closest pair split between `q` and `r` closer than
        `delta` in constant time"""
        closest = None
        # we only look at points with pivot - delta <= x <= pivot + delta
        l, r = pivot - delta, pivot + delta
        s = [p for p in py if p[0] >= l and p[0] <= r]
        len_s = len(s)
        # now we check the distance between each 7 consecutive points
        for i in range(len_s):
            j = i + 1
            while j < len_s and j <= i+7:
                dist = d(s[i], s[j])
                if dist < delta:
                    closest, delta = (s[i], s[j]), dist
                j += 1
        return delta, closest

    # Pivot point
    n = len(px) >> 1

    # if we're small enough, we can just return the smallest distance
    if n <= 4:
        delta, closest = None, None
        for i in range(len(px)):
            for j in range(i + 1, len(px)):
                dist = d(px[i], px[j])
                if delta is None or dist < delta:
                    closest, delta = (px[i], px[j]), dist
        return delta, closest

    # otherwise we recurse left and right, and find the closest split pairs
    qx = px[:n]
    qy = [p for p in py if p[0] <= qx[-1][0]]
    rx = px[n:]
    ry = [p for p in py if p[0] >= rx[0][0]]

    qc = closest_pair(qx, qy)
    rc = closest_pair(rx, ry)
    sc = closest_split_pair(qx[-1][0], min(qc[0], rc[0]))
    if sc[1] is not None:
        return min(qc, rc, sc)
    return min(qc, rc)
    

In [20]:
px, py = points(10)
closest_pair(px, py)

(10273.299177966152, ((63931, 53118), (54211, 56444)))

In [21]:
plt.plot([p[0] for p in px], [p[1] for p in px], 'ro')
bokeh.plotting.show(bokeh.mpl.to_bokeh(xkcd=True))